In [19]:
import eventlet
import os
import urllib.request as request
import pandas as pd
import regex
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

#Zawrzyj link do plików tesktowych gdzie kluczem będzie nazwa autora
book_files={
 "Mickiewicz": [
 "https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt",
 "https://wolnelektury.pl/media/book/txt/dziady-dziady-widowisko-czesc-i.txt",
 "https://wolnelektury.pl/media/book/txt/dziady-dziadow-czesci-iii-ustep-do-przyjaciol-moskali.txt",
 "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-pani-twardowska.txt",
 "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-powrot-taty.txt",
 "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-switez.txt",
 "https://wolnelektury.pl/media/book/txt/dziady-dziady-poema-dziady-czesc-iv.txt",
 ],
 "Sienkiewicz": [
 "https://wolnelektury.pl/media/book/txt/quo-vadis.txt",
 "https://wolnelektury.pl/media/book/txt/sienkiewicz-we-mgle.txt",
 "https://wolnelektury.pl/media/book/txt/potop-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/potop-tom-drugi.txt",
 "https://wolnelektury.pl/media/book/txt/potop-tom-trzeci.txt",
 ],
 "Orzeszkowa": [
 "https://wolnelektury.pl/media/book/txt/orzeszkowa-kto-winien.txt",
 "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-drugi.txt",
 "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-trzeci.txt",
 "https://wolnelektury.pl/media/book/txt/gloria-victis-dziwna-historia.txt",
 "https://wolnelektury.pl/media/book/txt/z-pozogi.txt",
 "https://wolnelektury.pl/media/book/txt/pani-dudkowa.txt",
 "https://wolnelektury.pl/media/book/txt/dymy.txt",
 "https://wolnelektury.pl/media/book/txt/syn-stolarza.txt",
 "https://wolnelektury.pl/media/book/txt/dobra-pani.txt",
 "https://wolnelektury.pl/media/book/txt/cnotliwi.txt",
 "https://wolnelektury.pl/media/book/txt/kilka-slow-o-kobietach.txt",
 "https://wolnelektury.pl/media/book/txt/patryotyzm-i-kosmopolityzm.txt",
 "https://wolnelektury.pl/media/book/txt/julianka.txt",
 ],
 "Prus": [
 "https://wolnelektury.pl/media/book/txt/lalka-tom-drugi.txt",
 "https://wolnelektury.pl/media/book/txt/lalka-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/antek.txt",
 "https://wolnelektury.pl/media/book/txt/katarynka.txt",
 "https://wolnelektury.pl/media/book/txt/prus-anielka.txt",
 "https://wolnelektury.pl/media/book/txt/prus-placowka.txt",
 
 ],
 "Reymont": [
 "https://wolnelektury.pl/media/book/txt/ziemia-obiecana-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/chlopi-czesc-pierwsza-jesien.txt",
 "https://wolnelektury.pl/media/book/txt/reymont-chlopi-zima.txt",
 "https://wolnelektury.pl/media/book/txt/chlopi-czesc-trzecia-wiosna.txt",
 "https://wolnelektury.pl/media/book/txt/chlopi-czesc-czwarta-lato.txt",
 ]
}


def fetch(url):
    file_path = os.path.join("data",os.path.basename(url))
    if os.path.exists(file_path):
        return None, None
    data = request.urlopen(url).read()
    return file_path, data

#Dla każdego linku pobierz zawartość i pobierz go do folderu data
for author in book_files:
    pool = eventlet.GreenPool()
    
    for file_path, data in pool.imap(fetch, book_files[author]):
        if file_path:
            with open(file_path, mode="wb") as f:
                f.write(data)
                

def preprocess_file(file_path=None, file_url=None):
    if not file_path and file_url:
        file_path = os.path.join("data",os.path.basename(file_url))
            
    #Pobierz zawartość plikut
    text = open(file_path,'rb').read().decode("utf-8").lower()
    
    text = regex.sub(u"[^ \n\p{Latin}\-'.?!]", " ",text) #Usuń znaki specjalne
    text = regex.sub(u"[ \n]+", " ", text) # Usuń spacje
    text = regex.sub(r"----- ta lektura.*","", text) # Usuń footer zawięrający streszenie po -----
    
    return [regex.sub(r"^ ","",l) for l in regex.split('\.|,|\?|!|:',text)]


#Zwróc w postaci data framu wartości autor : ilość lini
def get_book_df(document, author):
    return pd.DataFrame({
        'author': pd.Series(len(document)*[author]),
        'txt': pd.Series(document),
    })

#Złącz wszystkich autorów i liniami tekstów w jedną zmienną
book_lines_df = pd.concat([
    get_book_df(preprocess_file(file_url=url),author=author) 
        for author in book_files for url in book_files[author] 
])


#Utwórz zestaw danych do testów i treningu
train_df, test_df = train_test_split(
    book_lines_df, #data frame z autorami i liniami tekstów
    test_size=0.1, #90% treningu 10% testów
    stratify=book_lines_df['author'],
)

In [18]:
#W pierwszym kroku zbudujemy słownik składający się ze wszystkich słów występujących w tekście. 
#Następnie w każdej próbce (zdaniu) przypiszemy wektor długości takiej jak liczba unikalnych słów i na każdej pozycji 
#odpowiadającej określonemu słowu umieścimy liczbę odpowiadającą ilości wystąpień danego słowa w próbce.
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(train_df['txt']) #umieść w nim dane przeznaczone do treningu
sample_sentence = train_df.iloc[2]['txt']
print("Extracting vector for sentence '{}'".format(sample_sentence))
vectorizer.transform([sample_sentence])

Extracting vector for sentence 'wiedziała też o wszystkim'


<1x131523 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [20]:
X_train = vectorizer.transform(train_df['txt']) #zamień nasze teksty na zbiory wektorów wejściowych
X_test = vectorizer.transform(test_df['txt']) #zamień nasze teksty na zbiory wektorów wejściowych
# class_weight = przypisując im wagi odwrotnie proporcjonalne do częstotliwości występowania danej klasy | dual == wykorzystanie innego sposobu implementacji algorytmu regresji logistycznej,
model = LogisticRegression(class_weight='balanced',dual=True) 
model.fit(X_train,train_df['author']) #

C:\Users\Ragnus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ragnus\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Ragnus\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=True,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
model.score(X_test,test_df['author']) #Sprawdzenie jakości 

0.7647394136807818

In [22]:
from sklearn import metrics
#Zweryfikuj poprawność działania modelu pokazując parametry takie jak prezycja czy odwołanie
target = test_df['author']
predicted = model.predict(X_test)
print(metrics.classification_report(target,predicted))

              precision    recall  f1-score   support

  Mickiewicz       0.62      0.54      0.58       509
  Orzeszkowa       0.76      0.73      0.75      2218
        Prus       0.74      0.73      0.74      3104
     Reymont       0.75      0.76      0.76      2411
 Sienkiewicz       0.81      0.84      0.82      4038

    accuracy                           0.76     12280
   macro avg       0.74      0.72      0.73     12280
weighted avg       0.76      0.76      0.76     12280

